# granger 인과성 검정
    - Granger 인과검정의 영가설은 X가 Y에 영향을 미치지 않는다 이다. 즉 H0를 기각해야한다
    - 두 개의 시계열 데이터에서 한 변수(한 시계열)의 과거데이터와 다른 한 변수(시계열)의 과거데이터의 결합으로 그 변수(처음 시계열)를 선형 예측(linear regression)을 했을 때 다른 한 변수의 과거데이터로만 선형예측 한 것이 통계적으로 유의미하고 예측에 도움을 줬다면 그것을 그래인저 인과(Granger Causality)가 있다고 말한다.

In [3]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests
# df=[]

# df.replace(',', '', regex=True, inplace=True) 
# df = df.apply(pd.to_numeric, errors='coerce') # numeric 변환
maxlag= 14
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
   
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

# grangers_causation_matrix(df, variables = df.columns)

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

# plt.figure(figsize=(16,8))
# ax = sns.heatmap(df, annot=True, cmap='Blues')
# ax.xaxis.tick_top()
# plt.xticks(fontsize=12)
# plt.yticks(fontsize=12)
# plt.show()

# Cointegration Test
    - significant_yn이 모두 True이면서 dist가 가장 큰 그룹을 찾아보자.
    - 공적분 검정(혹은 요한슨 검정)은 다중 시계열 간 적분상 균형관계가 존재하는가를 판단한다
    - 상관관계는 시계열 적으로 연관성이 있는지 좀 더 정확한 관계 분석을 위해서 공적분 분석을 진행해보겠습니다.
    - 공적분 분석은 statsmodels 라이브러리를 이용해서 coint를 이용
    -  반환 값 중 p_value가 존재하는데요. 여기서의 귀무가설은 위에서도 언급했지만 서로 공적분 관계가 없다는 것이 귀무가설입니다
    - 따라서 p-value 값이 5%보다 작을 때는 이 귀무가설을 기각하여 서로 공적분 관계가 있다는 것을 알 수 있을겁니다.

In [6]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def calc_sig_dist(cols):
    out = coint_johansen(df[cols], 1, 1)
    stats = [round(x,2) for x in out.lr1]
    sigs = [round(x,2) for x in out.cvt[:, 1]]
    yns = [x>y for x,y in zip(stats,sigs)]
    dist = np.mean(np.array(stats) - np.array(sigs))
    

    print('stats: ',stats)                                            ## stats는 공적분 통계량
    print('sig-level: ',sigs)                                         ## sig-level은 유의수준 0.05에 해당하는 관측치
    print('significant_yn: ',[x>y for x,y in zip(stats,sigs)])        ## significant_yn은 유의 여부(장기 안정성), dist는 안정성의 강도를 의미한다.
    print('dist: ', round(dist,2))                                    ## dist는 안정성의 강도를 의미한다.

# Split Datasets (Train / Test)

#  Stationary Test
    - 영가설은 정상성을 띄지   --> H0를 기각해야 한다

In [9]:
from statsmodels.tsa.stattools import adfuller

# adf_sample = adfuller(df_train['samsung_ac'], autolag='AIC') # AIC가 가장 낮은 lag(시차)를 자동 선택
# adf_sample

# adf_df = pd.DataFrame(adf_sample[:4])
# adf_df.columns = ['samsung_ac']
# adf_df.index = ['stat','p_value','lag','observ']

# sig = pd.DataFrame(data={'samsung_ac':adf_sample[4]['5%']}, index=['5%'])
# adf_df = pd.concat([adf_df, sig], axis=0)
# adf_df

# VAR
    - AIC가 10 이상만 되어도 유의한 예측이라고 보기 어렵다.
    - 데이터 범위가 커서 예측 수준이 떨어질 수 있다
    - 일반적으로 AIC 기준 2.0 이하는 모델이 타당하다고 할 수 있다.
    -  전체 평균이 아니라 첫날 데이터를 0으로 기준을 잡고, 그 이후에는 첫날 대비 변화량을 보려줄 수 있도록 할 것이다.
    - 이때 다시한번 정상성 여부도 체크해준다.

- class statsmodels.tsa.vector_ar.var_model.VAR(endog, exog=None, dates=None, freq=None, missing='none')


VAR 클래스는 벡터 자기회귀(Vector Autoregression) 모델을 구현하는 statsmodels 라이브러리의 API입니다. VAR 클래스의 파라미터(attribute)에 대한 설명은 다음과 같습니다:

endog (ndarray): 모델의 종속 변수 데이터로 구성된 2차원 배열입니다. 각 열은 각각의 종속 변수를 나타내며, 행은 시간 인덱스를 나타냅니다.

exog (ndarray, optional): 모델의 독립 변수 데이터로 구성된 2차원 배열입니다. 각 열은 각각의 독립 변수를 나타내며, 행은 시간 인덱스를 나타냅니다. 기본값은 None입니다.

dates (array_like, optional): 날짜 또는 시간 인덱스를 나타내는 배열입니다. 기본값은 None이며, 인덱스가 없는 경우 None으로 설정됩니다.

freq (str, optional): 시계열의 빈도를 나타내는 문자열입니다. 'B', 'D', 'W', 'M', 'A', 등과 같은 주기를 지정할 수 있습니다. 기본값은 None입니다.

missing (str, optional): 결측치 처리 방법을 나타내는 문자열입니다. 'none', 'drop', 'raise' 중 하나를 선택할 수 있습니다. 기본값은 'none'으로 결측치를 처리하지 않음을 의미합니다.

p (int): 모델의 자기회귀 차수를 나타내는 정수입니다.

trend (str, optional): 모델의 추세 항 설정을 나타내는 문자열입니다. 'c' (상수), 'ct' (상수 및 추세), 'ctt' (상수, 추세, 추세의 제곱) 중 하나를 선택할 수 있습니다. 기본값은 'c'입니다.

method (str, optional): 모델 추정에 사용할 방법을 나타내는 문자열입니다. 'ols', 'yw' (Yule-Walker), 'burg' (Burg's method), 'ols-rew' (OLS with rewighting), 'burg-rew' (Burg's method with rewighting) 중 하나를 선택할 수 있습니다. 기본값은 'ols'입니다.

maxlags (int, optional): 모델 추정에 사용할 최대 자기회귀 차수를 제한하는 정수입니다. 기본값은 None으로, 자동으로 선택됩니다.

ic (str, optional): 최적의 모델 차수를 선택하는 정보 기준을 나타내는 문자열입니다. 'aic', 'bic', 'fpe', 'hqic' 중 하나를 선택할 수 있습니다. 기본값은 None으로, 정보 기준을 사용하지 않습니다.

trend_offset (int, optional): 추세의 시작 오프셋을 나타내는 정수입니다. 기본값은 1입니다.

seasonal (bool, optional): 계절성 구성 요소가 있는 경우 True로 설정하고, 없는 경우 False로 설정합니다. 기본값은 False입니다.

periods (int, optional): 계절성 주기의 길이를 나타내는 정수입니다. 기본값은 None으로, 계절성이 없는 경우 None으로 설정됩니다.

fir (int, optional): 계절성 주기의 FIR(유한 임펄스 응답) 길이를 나타내는 정수입니다. 기본값은 None으로, 계절성이 없는 경우 None으로 설정됩니다.

innov (None or ndarray, optional): 모델의 혁신(innovation) 항을 나타내는 배열입니다. 기본값은 None으로, 모델 학습을 통해 자동으로 추정됩니다.

위와 같이 VAR 클래스의 파라미터를 설정하고 모델을 초기화한 후에는 fit(), forecast(), plot_forecast() 등의 메서드를 사용하여 모델을 학습하고 예측 결과를 생성하고 시각화할 수 있습니다.

In [10]:
# from statsmodels.tsa.api import VAR

# var = VAR(df_train_)
#ㄴ

# model = var_norm.fit(14)
# model.summary()

In [11]:
# model.resid.corr()로 colleration matrix확인

# durbin_watson
### 잔차의 독립성 검정은 Durbin Watson 검정을 통해 수행

In [12]:
from statsmodels.stats.stattools import durbin_watson

# durbin_res = pd.DataFrame([model.resid.columns, 
#                            [round(x,2) for x in durbin_watson(model.resid)]]).T
# durbin_res.set_index([0])

- 범위 : 0~4
- 양의 상관관계 : 0에 수렴
- 음의 상관관계 : 4에 수렴
- 상관관계 없음 : 2에 수렴

# Forecast

In [17]:
# model.k_ar
# 14

In [16]:
# ins = df_train_norm.values[-model.k_ar:]
# ins
# 3개 변수에 대한 최근 14일치의 데이터다. 앞에서 우리는 최근 30일치를 기준으로 train dataset를 나눴기 때문에 30일치보다 앞선 14일치 데이터라고 보면 되겠다

In [18]:
# f = model.forecast(y=ins, steps=7)
# df_f = pd.DataFrame(f, columns=df_train_norm.columns)